# Bedrock with LangChain using a Prompt that includes Context

please refer the detail : https://github.com/aws-samples/amazon-bedrock-workshop/blob/main/01_Generation/02_contextual_generation.ipynb

## install dependencies

In [ ]:
# dependencies
!pip install boto3>=1.28.59
!pip install langchain
!pip install langchainhub
!pip install -U "anthropic[bedrock]"

## check bedrock availability

In [ ]:
def usage_demo():
    """
    Shows how to list the available foundation models.
    This demonstration gets the list of available foundation models and
    prints their respective summaries.
    """
    logging.basicConfig(level=logging.INFO)
    print("-" * 88)
    print("Welcome to the Amazon Bedrock demo.")
    print("-" * 88)

    bedrock_client = boto3.client(service_name="bedrock", region_name="us-east-1")

    wrapper = BedrockWrapper(bedrock_client)

    print("Listing the available foundation models.")

    try:
        for model in wrapper.list_foundation_models():
            print_model_details(model)
    except ClientError:
        logger.exception("Couldn't list foundation models.")
        raise

    print("Getting the details of an individual foundation model.")

    model_id = "amazon.titan-embed-text-v1"

    try:
        print_model_details(wrapper.get_foundation_model(model_id))
    except ClientError:
        logger.exception(f"Couldn't get foundation model {model_id}.")
        raise

In [ ]:
import json
import os
import sys
import logging
import boto3
import botocore

logger = logging.getLogger(__name__)

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock
from utils.BedrockWrapper import BedrockWrapper
from utils.GamePlayer import GamePlayer
from utils.GameAssistant import GameAssistant
from utils.GameMaster import GameMaster
from utils.PeTemplates import *
from utils import ParseJson, print_ww, Print, Info, Debug, Warn, Error
from utils import print_model_details
        
from botocore.exceptions import ClientError

usage_demo()
#boto3_bedrock = bedrock.get_bedrock_client()

#listModels = bedrock.list_foundation_models(byProvider='meta')

# print_ww("hello")
# Info("hello")

# GA = GameAssistant(template_assistant_role, 1000)
# print(GA.agent.prompt.template)

## Test Part

In [ ]:
import json
import os
import sys
import logging
import boto3
import botocore

logger = logging.getLogger(__name__)

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils.BedrockRuntimeWrapper import BedrockRuntimeWrapper, invoke
from botocore.exceptions import ClientError

client = boto3.client(service_name="bedrock-runtime", region_name="us-east-1")
wrapper = BedrockRuntimeWrapper(client)

mode_id = "anthropic.claude-3-sonnet-20240229-v1:0"

text_generation_prompt = "5+3等于多少？"
print("\n====claude3====\n")
invoke(wrapper, mode_id, text_generation_prompt)
print("\n====claude3 with stream response====\n")
try:
    async for completion in wrapper.invoke_claude3_with_response_stream(text_generation_prompt):
        print(completion, end="")

except ClientError:
    logger.exception("Couldn't invoke model %s", model_id)
    raise


In [ ]:
import boto3
from langchain_community.llms import Bedrock

bedrock = boto3.client('bedrock-runtime' , 'us-east-1')

MODEL_KWARGS = {
"anthropic.claude-3-sonnet-20240229-v1:0": {
        "temperature": 0, 
        "top_k": 250, 
        "top_p": 1, 
        "max_tokens_to_sample": 2**10 
}}

model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'
llm = Bedrock(model_id=model_id, model_kwargs=MODEL_KWARGS[model_id])
llm('tell me a joke')

In [ ]:
from anthropic import AnthropicBedrock

client = AnthropicBedrock(
    # Authenticate by either providing the keys below or use the default AWS credential providers, such as
    # using ~/.aws/credentials or the "AWS_SECRET_ACCESS_KEY" and "AWS_ACCESS_KEY_ID" environment variables.
    #aws_access_key="<access key>",
    #aws_secret_key="<secret key>",
    # Temporary credentials can be used with aws_session_token.
    # Read more at https://docs.aws.amazon.com/IAM/latest/UserGuide/id_credentials_temp.html.
    #aws_session_token="<session_token>",
    # aws_region changes the aws region to which the request is made. By default, we read AWS_REGION,
    # and if that's not present, we default to us-east-1. Note that we do not read ~/.aws/config for the region.
    aws_region="us-east-1",
)

message = client.messages.create(
    model="anthropic.claude-3-sonnet-20240229-v1:0",
    max_tokens=256,
    messages=[{"role": "user", "content": "Hello, world"}]
)
print(message.content)

In [ ]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: Apache-2.0
"""
Shows how to generate a message with Anthropic Claude (on demand).
"""
import boto3
import json
import logging

from botocore.exceptions import ClientError


logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

def generate_message(bedrock_runtime, model_id, system_prompt, messages, max_tokens):

    body=json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "system": system_prompt,
            "messages": messages
        }  
    )  

    response = bedrock_runtime.invoke_model(body=body, modelId=model_id)
    response_body = json.loads(response.get('body').read())
   
    return response_body

def main():
    """
    Entrypoint for Anthropic Claude message example.
    """

    try:

        bedrock_runtime = boto3.client(service_name='bedrock-runtime')

        model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'
        system_prompt = "Please respond."
        max_tokens = 1000

        # Prompt with user turn only.
        user_message =  {"role": "user", "content": "你是谁？"}
        messages = [user_message]

        response = generate_message (bedrock_runtime, model_id, system_prompt, messages, max_tokens)
        print("User turn only.")
        print(json.dumps(response, indent=2))

        # Prompt with both user turn and prefilled assistant response.
        #Anthropic Claude continues by using the prefilled assistant text.
        assistant_message =  {"role": "assistant", "content": "<emoji>"}
        messages = [user_message, assistant_message]
        response = generate_message(bedrock_runtime, model_id,system_prompt, messages, max_tokens)
        print("User turn and prefilled assistant response.")
        print(json.dumps(response, indent=4))

    except ClientError as err:
        message=err.response["Error"]["Message"]
        logger.error("A client error occurred: %s", message)
        print("A client error occured: " +
            format(message))

if __name__ == "__main__":
    main()



In [ ]:
from utils.PeTemplates import *
from langchain.agents import tool
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator

# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field
    
# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

print(prompt)

model = claude3_Sonnet
# And a query intended to prompt a language model to populate the data structure.
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "Tell me a joke."})
parser.invoke(output)

In [ ]:
from utils.CustomTools import *
from utils.PeTemplates import *
from langchain.agents import tool

# - 夜晚投票(狼人专属行动): WolfVote 参数: target=存活玩家
# - 夜晚查验(预言家专属行动): ProphetCheck 参数: target=存活玩家
# - 白天怀疑(所有玩家白天可选行动, 非投票): PlayerDoubt 参数: target=存活玩家 
# - 白天投票: PlayerVote 参数: target=存活玩家 
# - 白天讨论: Debate 参数: content=思考/理由 
# - 玩家信息: GetAllPlayersName 参数: 无 
# - 死亡遗言: DeathWords 参数: content=给予玩家线索
# - 玩家弃权: Pass 参数: 无 
# - 其他动作: Pass 参数: 无 

@tool
def WolfVote(target: str) -> str:
    """夜晚投票(狼人专属行动)"""
    return "WolfVote"

@tool
def ProphetCheck(target: str) -> str:
    """夜晚查验(预言家专属行动)"""
    return "ProphetCheck"

@tool
def PlayerDoubt(target: str) -> str:
    """白天怀疑(所有玩家白天可选行动, 非投票)"""
    return "PlayerDoubt"

@tool
def PlayerVote(target: str) -> str:
    """白天投票"""
    return "PlayerVote"

@tool
def Debate(target: str) -> str:
    """白天讨论"""
    return "Debate"

@tool
def GetAllPlayersName(target: str) -> str:
    """玩家信息"""
    return "GetAllPlayersName"

@tool
def DeathWords(content: str) -> str:
    """死亡遗言"""
    return "DeathWords"

@tool
def Pass(content: str) -> str:
    """玩家弃权,说明理由"""
    return "Pass"

 
# print(f"search.name:{search.name}")
# print(f"search.description:{search.description}")
# print(f"search.args:{search.args}")

# Define a list of tools
tools = [
    Tool(
        name = "WolfVote",
        func=WolfVote.run,
        description="夜晚投票(狼人专属行动)"
    ),
    Tool(
        name = "ProphetCheck",
        func=ProphetCheck.run,
        description="夜晚查验(预言家专属行动)"
    ),
    Tool(
        name = "PlayerDoubt",
        func=PlayerDoubt.run,
        description="白天怀疑(所有玩家白天可选行动, 非投票)"
    ),
    Tool(
        name = "PlayerVote",
        func=PlayerVote.run,
        description="白天投票"
    ),
    Tool(
        name = "Pass",
        func=Pass.run,
        description="玩家弃权,说明理由"
    )
]

# Using tools, the LLM chain and output_parser to make an agent
tool_names = [tool.name for tool in tools]

prompt = CustomPromptTemplate(
    template=template_werewolf_role.replace("{tool_names}", ",".join(tool_names)),
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps", "history", 'agent_scratchpad', 'tools', 'tool_names']
)
print(prompt)
# output_parser = CustomOutputParser()

llm = claude_instant_llm

# LLM chain consisting of the LLM and a prompt
# llm_chain = LLMChain(llm=llm, prompt=prompt)

agent = create_structured_chat_agent(llm, tools, prompt)

# agent = LLMSingleActionAgent(
#     llm_chain=llm_chain, 
#     output_parser=output_parser,
#     # We use "Observation" as our stop sequence so it will stop when it receives Tool output
#     # If you change your prompt template you'll need to adjust this as well
#     stop=["\nObservation:"], 
#     allowed_tools=tool_names
# )

# Initiate the agent that will respond to our queries
# Set verbose=True to share the CoT reasoning the LLM goes through
memory = ConversationBufferWindowMemory(k=2)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, memory=memory, verbose=False)

agent_executor.invoke({"input":"第一个晚上，你支持的玩家该如何行动？"})


In [1]:
from utils.GamePlayer import GamePlayer
from utils.GameMaster import GameMaster
from utils.PeTemplates import *
from utils import ParseJson, print_ww, Print, Info, Debug, Warn, Error

llm = claude_llm
GM = GameMaster(10, llm, False)
GM.ResetGame()
GM.RunGame()
GM.EndGame()

04/19/2024 03:42:17 AM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
04/19/2024 03:42:17 AM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
04/19/2024 03:42:17 AM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
04/19/2024 03:42:17 AM 	===== P1:存活,P2:存活,P3:存活,P4:存活,P5:存活,P6:存活,P7:存活,P8:存活 ResetGame =====
04/19/2024 03:42:17 AM P1 is 预言家
04/19/2024 03:42:17 AM P2 is 村民
04/19/2024 03:42:17 AM P3 is 狼人
04/19/2024 03:42:17 AM P4 is 村民
04/19/2024 03:42:17 AM P5 is 村民
04/19/2024 03:42:17 AM P6 is 狼人
04/19/2024 03:42:17 AM P7 is 村民
04/19/2024 03:42:17 AM P8 is 村民
04/19/2024 03:42:17 AM 	===== P1:存活,P2:存活,P3:存活,P4:存活,P5:存活,P6:存活,P7:存活,P8:存活 RunGame =====
04/19/2024 03:42:17 AM 	===== input_tokens: 0 output_tokens 0 ======
04/19/2024 03:42:17 AM 	===== elapsed_time: 0.00737452507019043 ======
04/19/2024 03:42:17 AM 	===== NIGHT 1 PreAction ======
04/19/2024 03:42:17 AM 	===== NIGHT 1 DoAction ======
04/19/2024 03:42:17 AM 		===== D

 第一个夜晚所有的行动都是随机的,作为预言家我需要随机选择一个人去查验。

{"action": "ProphetCheck", "target": "P2"}

04/19/2024 03:42:21 AM 		===== DoAction NIGHT 1 P1 ======
04/19/2024 03:42:21 AM [玩家P1于时间NIGHT 1, 执行动作为:{'action': 'ProphetCheck', 'target': 'P2'}]P2:村民
04/19/2024 03:42:21 AM 
 [ROUND ACTION]={'time': 'NIGHT 1', 'player': 'P1', 'status': 1, 'role': '预言家', 'character': '独立思考', 'response': ['{"action": "ProphetCheck", "target": "P2"}']}

04/19/2024 03:42:21 AM 		===== DoMemory NIGHT 1 P3 ======
04/19/2024 03:42:21 AM 		===== DoReflect NIGHT 1 P3 ======
04/19/2024 03:42:21 AM 		===== DoAnswer NIGHT 1 P3 ======
04/19/2024 03:42:21 AM 	QUESTION: 现在是NIGHT 1,你支持的玩家是P3(狼人身份,本阵营为:P3,P6).目前场上玩家:P1:存活,P2:存活,P3:存活,P4:存活,P5:存活,P6:存活,P7:存活,P8:存活(逗号为分割符).现在是第1天夜晚，该如何行动?


 第一天晚上我的行动是随机的,不好判断其他人的身份。我的建议是:

{"action": "WolfVote", "target": "P7"}

第一天晚上随便投票一个人,不要暴露身份。

04/19/2024 03:42:30 AM 		===== DoAction NIGHT 1 P3 ======
04/19/2024 03:42:30 AM 
 [ROUND ACTION]={'time': 'NIGHT 1', 'player': 'P3', 'status': 1, 'role': '狼人', 'character': '组织者', 'response': ['{"action": "WolfVote", "target": "P7"}']}

04/19/2024 03:42:30 AM 		===== DoMemory NIGHT 1 P6 ======


...

04/19/2024 03:43:54 AM 		===== DoReflect DAY 2 P3 ======
04/19/2024 03:43:54 AM 		===== DoAnswer DAY 2 P3 ======
04/19/2024 03:43:54 AM 	QUESTION: 现在是DAY 2,你支持的玩家是P3(狼人身份,本阵营为:P3,P6).目前场上玩家:P1:存活,P2:存活,P3:存活,P4:存活,P5:存活,P6:存活,P7:淘汰,P8:存活(逗号为分割符)..开始一步一步进行推理，为下一轮行动准备.


 既然P7死亡遗言怀疑我P3,那我需要先进行辩解。我的思考是:

{"action": "Debate", "content": "我不是狼人,P7死亡遗言纯属无端诬陷。"}

接下来还需要引导其他人的注意力,转移他们的视线。我建议:

{"action": "PlayerDoubt", "target": "P2"} 

通过怀疑P2,可以转移其他人对我的注意力,同时也能观察P2的反应,看他是否紧张。

04/19/2024 03:44:21 AM 		===== DoAnswer DAY 2 P3 ======
04/19/2024 03:44:21 AM 	QUESTION: 现在是DAY 2,你支持的玩家是P3(狼人身份,本阵营为:P3,P6).目前场上玩家:P1:存活,P2:存活,P3:存活,P4:存活,P5:存活,P6:存活,P7:淘汰,P8:存活(逗号为分割符).现在是第2天白天的讨论环节，该如何行动?


 {"action": "Debate", "content": "我不是狼人,P7的死亡遗言纯属

KeyboardInterrupt: 